# Elaborato Teorie e Tecniche del Riconoscimento

Nel 2016 la [National Library of Medicine](https://www.nlm.nih.gov/) ha proposto 
la sfida ["Pill Image Recognition"](https://pir.nlm.nih.gov/challenge)

L'obiettivo è quello di implementare un metodo automatico per la classificazione 
di pillole a partire da un'immagine in modo da prevenire possibili errori umani 
che possano causare rischi per la salute.

## Indice
- todo


Per le configurazioni necessarie per il funzionamento dello script è possibile 
modificare i dati nella cella seguente. 
N.B. È sempre necessario eseguire questa cella prima di qualunque altra per non 
incorrere in errori.



In [2]:
from pathlib import Path

# dataset directory: it should contain the images and the xml file 
DATASET_DIR = Path('dataset/merge')

# directory of backgrounds: it should contains the background samples 
# used to generate images 
BACKGROUNDS_DIR = Path('utils/backgrounds')

# path to the folder that is supposed to contain the datasets
# saved in npy format
SAVED_DATA_DIR = Path('saved_data')

# number of image that should be generated foreach already segmented 
# image in the dataset
AUTO_GENERATED_NUM = 10

# number of feature's vectors foreach batch
FEATURE_PER_BATCH = 500

# This is the number of feature that the script will use 
# N.B. It is NOT a configurable parameter. It is constant and 
# it must not be changed.
FEATURES_NUMBER = 10 

# maximun number of jobs to use for parallelization
MAX_JOBS = 12

## Dataset

Il dataset fornito per la sfida non include delle informazioni sulle immagini,
mentre il dataset disponibile sul server FTP della NLM include quasi 1TB di immagini
consumer di pillole con relative informazioni in formato XML.

Abbiamo scelto di creare il nostro dataset considerando un numero di classi 
limitato (codice NDC9) e filtrando le immagini sulla base del rating di alcuni 
parametri (shadow, background, lighting).

Per arricchire il dataset abbiamo implementato delle tecniche di data 
augmentation: sfruttando delle immagini con pillole già segmentate abbiamo introdotto 
una variabilità sul background, presenza di ombre e posa della pillola.

<img src="images/data_aug.png"/>

## Pipeline

<img src="images/seg.png"/>

### Segmentazione

La segmentazione è basata sulla trasformazione di Watershed, ottenuti i superpixel viene calcolato un Region Adjacency Graph (RAG) basato sui colori, in seguito le regioni vengono progressivamente unite in base alla similarità del colore. 

### ROI

Per capire qual è la nostra ROI utilizziamo delle euristiche sui parametri delle regioni ottenute (solidity, similarità ad un'ellisse, dimensione dell'area).

### Estrazione delle feature

Abbiamo deciso di utilizzare come feature i momenti di Hu per la shape e i valori RGB del colore dominante nella regione, per un totale di 10 feature. Per ottenere il colore dominante applichiamo un KMeans sulla ROI ottenendo al più 3 cluster, in questo modo uniamo eventuali regioni di colore che si sono formate durante il merge dei nodi del RAG e dovremmo avere una più netta separazione tra pillola e background.

In [4]:
import numpy as np 
import xml.etree.ElementTree as ET
from skimage.io import imread
from skimage import img_as_float
from skimage.transform import resize
from joblib import Parallel, delayed
from time import time
from tqdm import tqdm

from pillclassification.feature_extraction import feature_extraction
from pillclassification.functions import crop_center, generate_image
from utils.utils import tqdm_joblib
import warnings
warnings.filterwarnings('ignore')

# Loading all images filenames
images_dir = DATASET_DIR
filenames = [x for x in images_dir.iterdir() if x.suffix != '.xml']
bg_dir = BACKGROUNDS_DIR
bgs = [x for x in bg_dir.iterdir()]

# Vars
samples_num = len(filenames)

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

batches = list(chunks(filenames, FEATURE_PER_BATCH))

# Calculating labels 
try:
    tree = ET.parse(images_dir / 'images.xml')
except ET.ParseError:
    print('Parse error on {}'.format(images_dir / 'images.xml'))
    exit(-1)

se = list(tree.getroot())[0]

labels_set = set()
segmented = 0

for e in list(se):
    labels_set.add(e.find('NDC9').text)
    layout = e.find('Layout')
    if layout is not None and layout.text == "MC_C3PI_REFERENCE_SEG_V1.6":
        segmented += 1

labels = sorted(list(labels_set))
class_num = len(labels)

# Number of samples that will be used (also generated images)
final_samples = segmented * AUTO_GENERATED_NUM + len(filenames) - segmented

def extract_features(f):
    # loading the image 
    try:
        img = imread(f)
    except ValueError as e:
        return None
    
    
    features = np.ndarray((0,10))
    labels_ = []
    
    # img is rgba
    if img.shape[-1] == 4:
        for _ in range(AUTO_GENERATED_NUM):
            generated = generate_image(img, bgs[np.random.randint(0, len(bgs))])
            try:
                hu, rgb_val = feature_extraction(generated)
            except ValueError:
                continue
                
            label = -1
            for e in list(se):
                if e.find('File').find('Name').text == f.name:
                    label = labels.index(e.find('NDC9').text)
                    break
            features = np.append(features, [np.append(hu, rgb_val)], axis=0)
            labels_ = np.append(labels_, label)
        return features, labels_
    
    # cropping in the center
    img = crop_center(img, crop_scale=0.6)

    # rescaling with fixed width
    width = 600
    img = resize(img, (int(img.shape[0] * (width / img.shape[1])), width), anti_aliasing=True)

    # the img must be in float format 
    img = img_as_float(img)
    
    # feature extraction
    try:
        hu, rgb_val = feature_extraction(img)
    except ValueError:
        return None

    label = -1
    for e in list(se):
        if e.find('File').find('Name').text == f.name:
            label = labels.index(e.find('NDC9').text)
            break
    
    features = np.append(features, [np.append(hu, rgb_val)], axis=0)
    labels_ = np.append(labels_, label)
    return features, labels_

# creating dirs if they do not exists
(SAVED_DATA_DIR / 'x').mkdir(parents=True, exist_ok=True)
(SAVED_DATA_DIR / 'y').mkdir(parents=True, exist_ok=True)

# Extracting features
with tqdm_joblib(tqdm(desc="Feature extraction", total=len(filenames))) as progress_bar:
    with Parallel(n_jobs=MAX_JOBS) as parallel:
        for idx, batch in enumerate(batches):
            x_data = np.ndarray((0, FEATURES_NUMBER))
            y_data = np.array([], dtype=np.int32)
            for _, r in enumerate(parallel(delayed(extract_features)(f) for f in batch)):
                if r is not None:
                    x_data = np.concatenate((x_data, r[0]))
                    y_data = np.concatenate((y_data, r[1]))

            # Saving the features as npy file
            np.save(SAVED_DATA_DIR / 'x' / str(idx), x_data)
            np.save(SAVED_DATA_DIR / 'y' / str(idx), y_data)


Feature extraction: 100%|██████████| 742/742 [34:43<00:00,  2.81s/it]


In [3]:
from collections import Counter
import sys
import numpy as np 

# Load all features

x_data = np.ndarray((0, FEATURES_NUMBER))
y_data = np.array([], dtype=np.int32)

for x in (SAVED_DATA_DIR / 'x').iterdir():
    if x.is_file() and x.suffix == '.npy':
        y = SAVED_DATA_DIR / 'y' / x.name 
        
        if not y.exists():
            raise RuntimeError("Can't find y data for " + x.name + ' data file')
        
    try:
        x_data = np.concatenate((x_data, np.load(x)))
        y_data = np.concatenate((y_data, np.load(y)))
    except FileNotFoundError as e:
        print('cannot load data for the batch {}.'.format(str(x.stem)) + 
              'Try to extract the features again', file=sys.stderr)
        raise e

# Clear data from nan
idx = np.isnan(x_data[:])
idx = np.where(np.any(idx == True, axis=1))

for i in idx:
    x_data = np.delete(x_data, i, 0)
    y_data = np.delete(y_data, i, 0)
    
# Divide dataset in train + test

idx_sort = np.argsort(y_data)

y_sorted = y_data.take(idx_sort, 0)
x_sorted = x_data.take(idx_sort, 0)

j = 0

idx_train = []

cnt = Counter() 

for y in y_sorted:
    cnt[int(y)] += 1

for i in range(len(cnt)):
    train_n = int(cnt[i] * 0.9)
    for e in y_sorted[j:]:
        if e == i:
            break
        j += 1
    idx_train.extend(range(j, train_n + j))

idx_test = np.delete(list(range(len(y_data))), idx_train, 0)
X = x_sorted
Y = y_sorted

### Rilevamento delle anomalie

Per 'pulire' il dataset utilizziamo l'algoritmo di Isolation Forest. L'algoritmo isola le osservazioni selezionando randomicamente una feature e un valore compreso tra il massimo e il minimo della feature selezionata. Dato che un partizionamento può essere rappresentanto da una struttura ad albero, il numero di partizionamenti richiesti per isolare un campione sono al massimo la lunghezza del percorso dalla radice al nodo di terminazione. Questa lunghezza di percorso, mediata su una foresta di alberi random, diventa la nostra funzione di decisione.

Quando una foresta di alberi random produce un percorso più corto per un campione, allora quasi sicuramente è un'anomalia.

In [4]:
# Outlier detection
from sklearn.ensemble import IsolationForest

iso = IsolationForest(warm_start=True)
yhat = iso.fit_predict(X[idx_train])

# Select all rows that are not outliers
mask = yhat != -1
X_train = X[idx_train]
y_train = Y[idx_train]
X_train, y_train = X_train[mask, :], y_train[mask]


yhat = iso.fit_predict(X[idx_test])

mask = yhat != -1
X_test = X[idx_test]
y_test = Y[idx_test]
X_test, y_test = X_test[mask, :], y_test[mask]

# Summarize the shape of the updated training dataset
print('Total: {}\nTrain: {}\nTest: {}'.format(X.shape[0], X_train.shape[0], X_test.shape[0]))

Total: 3403
Train: 2314
Test: 273


### Classificazione delle feature

Abbiamo provato a mettere a confronto quattro classificatori:

- Regressione logistica:
- SVM lineare:
- SVM non lineare:
- Gradient boosting:

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
from sklearn.metrics import mean_absolute_error

from tqdm import tqdm
from joblib import Parallel, delayed
from utils.utils import tqdm_joblib
from tqdm import tqdm_notebook

def get_name(estimator):
    name = estimator.__class__.__name__
    if name == 'Pipeline':
        name = [get_name(est[1]) for est in estimator.steps]
        name = ' + '.join(name)
    return name


# list of (estimator, param_grid), where param_grid is used in GridSearchCV
classifiers = [
    (make_pipeline(
        KBinsDiscretizer(encode='onehot'),
        LogisticRegression(random_state=0)), {
            'kbinsdiscretizer__n_bins': np.arange(2, 10),
            'logisticregression__C': np.logspace(-2, 7, 10),
        }),
    (make_pipeline(
        KBinsDiscretizer(encode='onehot'),
        LinearSVC(random_state=0, tol=1e-6, max_iter=10000)), {
            'kbinsdiscretizer__n_bins': np.arange(2, 10),
            'linearsvc__C': np.logspace(-2, 7, 10),
        }),
    (GradientBoostingClassifier(n_estimators=100, random_state=0), {
        'learning_rate': np.logspace(-4, 0, 10)
    }),
    (SVC(random_state=0, tol=1e-6, max_iter=10000), {
        'C': np.logspace(-2, 7, 10)
    }),
]

names = [get_name(e) for e, g in classifiers]
clfs = []

scores = dict()
maes = dict()

def evaluate_clf(estimator, param_grid, n_jobs):
    clf = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs=n_jobs)
    with ignore_warnings(category=ConvergenceWarning):
        clf.fit(X_train, y_train)
    
    yhat = clf.predict(X_test)
    mae = mean_absolute_error(y_test, yhat)
    score = clf.score(X_test, y_test)

    return get_name(estimator), score, mae



with tqdm_joblib(tqdm(desc="Classifiers evaluation", total=len(classifiers))) as progress_bar:
    jobs = len(classifiers) if len(classifiers) < MAX_JOBS else MAX_JOBS

    if MAX_JOBS - jobs >= len(classifiers):
        jobs_for_grid = (MAX_JOBS - jobs) //  len(classifiers)
    else:
        jobs_for_grid = 1
    with Parallel(n_jobs=MAX_JOBS) as parallel:
        for name, score, mae in parallel(delayed(evaluate_clf)(estimator=e, param_grid=p, n_jobs=jobs_for_grid) for e, p in classifiers):
            print('\n{} MAE: {:.3f} Score: {:.3f}'.format(name, mae, score), end='')


# for est_idx, (name, (estimator, param_grid)) in enumerate(zip(names, classifiers)):
#     clf = GridSearchCV(estimator=estimator, param_grid=param_grid, n_jobs = -1)
#     clfs.append(clf)
#     with ignore_warnings(category=ConvergenceWarning):
#         clf.fit(X_train, y_train)
    
#     # Evaluate the model
#     yhat = clf.predict(X_test)
    
#     # Evaluate predictions
#     mae = mean_absolute_error(y_test, yhat)
    
#     score = clf.score(X_test, y_test)
#     print('{} MAE: {:.3f} Score: {:.3f}'.format(name, mae, score))

Classifiers evaluation: 100%|██████████| 4/4 [00:07<00:00,  1.91s/it]


KeyboardInterrupt: 

In [12]:
print(scores, mees)

NameError: name 'mees' is not defined

In [ ]:
i = 0
print('Predicted: {}, Truth: {}'.format(clfs[4].predict([X[idx_test][i]])[0], Y[idx_test][i]))

In [ ]:
# print(x[0])
# print(y_data)

import matplotlib.pyplot as plt
# r = lin_clf.predict([x[0]])
# print(lin_clf.decision_function([x[0]]))

print(lin_clf.score(X[idx_test], Y[idx_test]))

In [ ]:
from mlxtend.plotting import plot_decision_regions
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.svm import SVC

# Plotting decision regions
fig, ax = plt.subplots(figsize=(10,10))
# Decision region for feature 3 = 1.5
value = 2
# Plot training sample with feature 3 = 1.5 +/- 0.75
width = 0.75

feature_values = {i:value for i in range(2, 10)}  
feature_width = {i:value for i in range(2, 10)}  

plot_decision_regions(X, Y.astype(np.integer), clf=lin_clf,
                      filler_feature_values=feature_values,
                      filler_feature_ranges=feature_width,
                      res=0.02, ax=ax)
ax.set_xlabel('Feature 1')
ax.set_ylabel('Feature 2')
ax.set_title('Feature 3 = {}'.format(value))

plt.show()